# BCS Testing Breakdown

In [1]:
# imports
import numpy as np
import math  
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import qmc

import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools
from PyUQTk.utils.func import *
import PyUQTk.uqtkarray as uqtkarray

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


## Inputs

In [2]:
nord = 15           # Order of the final PCE
pc_type = "LU"     # Polynomial type

ndim = 4
upit=12

pc_alpha = 0.0     # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs

c_opt='seq'
sigma = np.array([1.e-08]) # inital noise variance; updated in BCS
model='LU'
nSam=10000

eta=1e-9

## Model for BCS

In [3]:
pc_model = uqtkpce.PCSet("NISPnoq", nord-upit, ndim, pc_type, pc_alpha, pc_beta)

## Setting Training and Testing

In [4]:
# polynomial goal
poly = uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha, pc_beta)
npce_full=poly.GetNumberPCTerms()

# sequence of increasing integers
if (c_opt=="seq"):
    coef=np.array([1/(i+1) for i in range(npce_full)])
    
# sequence of increasing integers alternating with zero    
elif (c_opt=="alt"):
    coef=np.zeros(npce_full)
    for i in range(npce_full):
        if (i%2==0):
            coef[i]=1/(i+1)

# Training Data
nTest=int(npce_full*2)
rng = qmc.LatinHypercube(d=ndim, seed=43)

x_tr=2*rng.random(n=nTest)-1 #[-1,1]
if (model=='LU'):
    y_tr=pce_tools.UQTkEvaluatePCE(poly, coef, x_tr)
else:
    y_tr=func(x_tr,model,np.ones(ndim+1))
    
# Testing Data
x_test=2*rng.random(n=nSam)-1

if (model=='LU'):
    y_test=pce_tools.UQTkEvaluatePCE(poly, coef, x_test)
else:
    y_test=func(x_test, model, np.ones(ndim+1))

## BCS inputs

In [5]:
lambda_init=np.array([])
adaptive = 0 # Flag for adaptive implementation, using a generative basis, set to 0 or 1
optimal = 1  # Flag for optimal implementation, rigorous implementation of adaptive CS, set to 0 or 1
scale = 0.1  # Diagonal loading parameter; relevant only in adaptive, non-optimal implementation

In [6]:
print("Our starting basis has", pc_model.GetNumberPCTerms(), "terms")

Our starting basis has 35 terms


## Iteration 0

In [7]:
# UQTk array for sigma
sig_uqtk=uqtkarray.dblArray1D(sigma.size)
sig_uqtk=uqtkarray.numpy2uqtk(np.asfortranarray(sigma))

# UQTk array for lambda_init
lam_uqtk=uqtkarray.dblArray1D(sigma.size)
lam_uqtk=uqtkarray.numpy2uqtk(np.asfortranarray(lambda_init))

#UQTk array for samples - [nqp, ndim]
sam_uqtk=uqtkarray.dblArray2D(y_tr.shape[0], ndim)
sam_uqtk=uqtkarray.numpy2uqtk(np.asfortranarray(x_tr))

# UQTk array for function f_evaluations
y = uqtkarray.dblArray1D()
y = uqtkarray.numpy2uqtk(np.asfortranarray(y_tr))

print("After the 0th iteration")
weights, used = pce_tools.evalbcs(pc_model, y, sam_uqtk, sig_uqtk, eta, lam_uqtk, adaptive, optimal, scale)

After the 0th iteration
BCS has selected 35 basis terms out of 35


In [9]:
import PyUQTk.utils.multiindex as uqtkmi
if(upit>=0):
    for it in range(upit):
        print("After Iteration", it+1)
        
        # uqtk array for multiindex
        mi_uqtk = uqtkarray.intArray2D(pc_model.GetNumberPCTerms(),ndim)
        # retrieve multiindex
        pc_model.GetMultiIndex(mi_uqtk)
        #uqtk array new multi index
        used_mi_uqtk=uqtkarray.intArray2D()
        # calculate new multiindex from the rows that are being used
        uqtkarray.subMatrix_row_int(mi_uqtk, used, used_mi_uqtk)
        # convert to numpy array
        used_mi_np=uqtkarray.uqtk2numpy(used_mi_uqtk)

        # grow the multiindex
        mi_next = np.array(uqtkmi.mi_addfront_cons(used_mi_np), dtype=object)[0]
        print(mi_next.shape[0]-used.XSize(), "terms added; new multiindex has", mi_next.shape[0], "terms")
        
        # convert to uqtk
        mi_next_uqtk = uqtkarray.intArray2D(mi_next.shape[0], mi_next.shape[1])
        for i in range(mi_next.shape[0]):
            for j in range(mi_next.shape[1]):
                mi_next_uqtk.assign(i, j, mi_next[i][j])

        pc_model=uqtkpce.PCSet("NISPnoq", mi_next_uqtk, pc_model.GetPCType(), pc_model.GetAlpha(), pc_model.GetBeta())

        weights, used = pce_tools.evalbcs(pc_model, y, sam_uqtk, sig_uqtk, eta, lam_uqtk, adaptive, optimal, scale)

After Iteration 1
35 terms added; new multiindex has 70 terms
BCS has selected 48 basis terms out of 70
After Iteration 2
32 terms added; new multiindex has 80 terms
BCS has selected 52 basis terms out of 80
After Iteration 3
30 terms added; new multiindex has 82 terms
BCS has selected 52 basis terms out of 82
After Iteration 4
27 terms added; new multiindex has 79 terms
BCS has selected 52 basis terms out of 79
After Iteration 5
29 terms added; new multiindex has 81 terms
BCS has selected 52 basis terms out of 81
After Iteration 6
29 terms added; new multiindex has 81 terms
BCS has selected 52 basis terms out of 81
After Iteration 7
31 terms added; new multiindex has 83 terms
BCS has selected 53 basis terms out of 83
After Iteration 8
28 terms added; new multiindex has 81 terms
BCS has selected 52 basis terms out of 81
After Iteration 9
30 terms added; new multiindex has 82 terms
BCS has selected 53 basis terms out of 82
After Iteration 10
31 terms added; new multiindex has 84 terms
B

In [10]:
# Create Numpy array of PC coefficients
c_k=np.zeros(pc_model.GetNumberPCTerms())
for i in range(used.XSize()):
    pos = used[i]
    c_k[pos]=weights[i]

## Training Error

In [11]:
pce_evals_train=pce_tools.UQTkEvaluatePCE(pc_model, c_k, x_tr)

MSE = np.square(np.subtract(y_tr, pce_evals_train)).mean()
RMSE=math.sqrt(MSE)
NRMSE=RMSE/np.linalg.norm(y_test)
print("RMSE:", RMSE)
print("NRMSE:", NRMSE)

RMSE: 0.021386798485630215
NRMSE: 0.00019667299954636018


## Testing Error

In [12]:
### Testing

pce_evals_test=pce_tools.UQTkEvaluatePCE(pc_model, c_k, x_test)

# metrics
MSE_test = np.square(np.subtract(y_test,pce_evals_test)).mean()
NRMSE=math.sqrt(MSE_test)/np.linalg.norm(y_test)
print("RMSE:", math.sqrt(MSE_test))
print("NRMSE:", math.sqrt(MSE_test)/np.linalg.norm(y_test))

RMSE: 0.02467478784780179
NRMSE: 0.00022690934982428035
